In [1]:
# 나이키 신발 신상 정보
# nike.co.kr
# NEW RELEASE - NEW FOR MAN - 신발

# 스크롤 페이징이 구현되어 있슴을 확인하자
# '개발자 도구' 에서 각 페이지마다 '어떠한 URL' 로 '무엇'을 request-response 하는지 확인하자.

# 최신 n 페이지의 '제품명', '가격', '페이지 url' 을 크롤링 하기
# 여러 페이징한 내용을 합쳐서, 가격별 정렬하고
# 그 결과물을  엑셀 저장하기




#  스크롤 페이징 분석

In [3]:
import os
import requests
from bs4 import BeautifulSoup
import urllib.parse as urlparser

page = 2
url = f"https://www.nike.com/kr/ko_kr/w/men/fw?page={2}&pageSize=40&lineSize=5&_=1595420474483"
# user-agent 값 꼭 필요하다
# 이 값이 없으면 response 200 이어도 데이터가 없다...
header = {
    "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36"
}

response = requests.get(url, headers = header)
response

<Response [200]>

In [4]:
dom = BeautifulSoup(response.text, "html.parser")
dom

<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no, viewport-fit=cover" name="viewport"/>
<meta content="breeze commerce" name="author"/>
<link href="//www.google.com" rel="dns-prefetch"/>
<link href="//www.google-analytics.com" rel="dns-prefetch"/>
<meta content="나이키" id="f_site_name" name="f_site_name" property="og:site_name"/>
<link href="https://static-breeze.nike.co.kr/kr/ko_kr/cmsstatic/theme/52/android-icon-36x36.png" rel="icon" sizes="16x16 32x32"/>
<link href="https://static-breeze.nike.co.kr/kr/ko_kr/cmsstatic/theme/52/android-icon-36x36.png" rel="shortcut icon" sizes="16x16 32x32" type="image/x-icon"/>
<title>신발 - 나이키</title>
<title>나이키</title>
<link href="https://static-breeze.nike.co.kr/kr/ko_kr/css/common_new-2.css?20200718073600" rel="stylesheet"/>
<sc

In [5]:
items = dom.select(".contents-body .item-list-wrap div.a-product") # 이벤트가 아닌 상품 정보는 a-product라는 class를 가지고 있음
len(items)

40

In [6]:
"""
제품명: .a-product-info .product-info .product-display-name span
가격: .a-product-info .product-price p
url: .a-product-image a
"""




'\n제품명: .a-product-info .product-info .product-display-name span\n가격: .a-product-info .product-price p\nurl: .a-product-image a\n'

In [7]:
items[0].select_one(".a-product-info .product-info .product-display-name span")

<span class="item-title">조던 CRATER</span>

In [8]:
items[0].select_one(".a-product-info .product-price p")

<p class="product-display-price">179,000 원</p>

In [9]:
items[0].select_one(".a-product-image a").attrs.get("href")
# https://www.nike.com/ 추가

'/kr/ko_kr/t/men/fw/basketball/CT0712-001/usau33/jordan-crater'

In [10]:
result = [
    {
        "제품명" : item.select_one(".a-product-info .product-info .product-display-name span").text.strip(),
        "가격" : float(item.select_one(".a-product-info .product-price p").text.replace(',', '')[:-2]) ,
        "url" : "https://www.nike.com/" + item.select_one(".a-product-image a").attrs.get("href")
    }
    
    for item in items
]
result

[{'제품명': '조던 CRATER',
  '가격': 179000.0,
  'url': 'https://www.nike.com//kr/ko_kr/t/men/fw/basketball/CT0712-001/usau33/jordan-crater'},
 {'제품명': '나이키 에어맥스 270 리액트',
  '가격': 179000.0,
  'url': 'https://www.nike.com//kr/ko_kr/t/men/fw/nike-sportswear/CT1280-100/qxrq51/air-max-270-react'},
 {'제품명': '나이키 와플 레이서 2X',
  '가격': 119000.0,
  'url': 'https://www.nike.com//kr/ko_kr/t/women/fw/nike-sportswear/CK6647-001/rjww83/w-nike-waffle-racer-2x'},
 {'제품명': '나이키 에어 포스 1 ‘07 LV8',
  '가격': 97300.0,
  'url': 'https://www.nike.com//kr/ko_kr/t/men/fw/nike-sportswear/CD0888-001/uwik93/air-force-1-07-lv8-3'},
 {'제품명': '나이키 에어맥스 95 에센셜',
  '가격': 125300.0,
  'url': 'https://www.nike.com//kr/ko_kr/t/men/fw/nike-sportswear/CI3705-002/sgtt18/nike-air-max-95-essential'},
 {'제품명': '나이키 에어 포스 1 리액트',
  '가격': 159000.0,
  'url': 'https://www.nike.com//kr/ko_kr/t/men/fw/nike-sportswear/CT1020-002/ivym25/air-force-1-react'},
 {'제품명': '에어 조던 5 레트로',
  '가격': 229000.0,
  'url': 'https://www.nike.com//kr/ko_kr/t/men/

In [11]:
import re # regex 정규표현식 라이브러리
int(re.sub('[^0-9]', '', '100,000 원'))

100000

In [12]:
# 한 페이지 크롤링

In [13]:
import os
import requests
from bs4 import BeautifulSoup
import urllib.parse as urlparser


def nike_page(page = 1):
    url = f"https://www.nike.com/kr/ko_kr/w/men/fw?page={page}&pageSize=40&lineSize=5&_=1595420474483"
    header = {
        "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36"
    }
    response = requests.get(url, headers = header)
    dom = BeautifulSoup(response.text, "html.parser")
    items = dom.select(".contents-body .item-list-wrap div.a-product")
    result = [
        {
            "제품명" : item.select_one(".a-product-info .product-info .product-display-name span").text.strip(),
            "가격" : int(re.sub("[^0-9]", "", item.select_one(".a-product-info .product-price p").text.strip())) ,
            "url" : "https://www.nike.com" + item.select_one(".a-product-image a").attrs.get("href")
        }

        for item in items
    ]
    
    return result

In [26]:
result = nike_page(3)

In [27]:
import pandas as pd
df = pd.DataFrame(result)
df.index = df.index + 1

In [29]:
df.to_excel("abc.xlsx")

In [30]:
df

,제품명,가격,url
1,르브론 17 로우,189000,https://www.nike.com/kr/ko_kr/t/men/fw/basketb...
2,PG 4 EP,129000,https://www.nike.com/kr/ko_kr/t/men/fw/basketb...
3,PG 4 EP,129000,https://www.nike.com/kr/ko_kr/t/men/fw/basketb...
4,나이키 에어맥스 알파 세비지 2,129000,https://www.nike.com/kr/ko_kr/t/men/fw/men-tra...
5,나이키 에어맥스 알파 세비지 2,129000,https://www.nike.com/kr/ko_kr/t/men/fw/men-tra...
6,나이키 에어 베이퍼맥스 360,259000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
7,나이키 에어맥스 270 리액트,179000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
8,나이키 SB 블레이저 로우 GT,89000,https://www.nike.com/kr/ko_kr/t/adult-unisex/f...
9,나이키 블레이저 미드 '77 빈티지,119000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
10,나이키 에어맥스 EXOSENSE,139000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...


# 실습 (원하는 페이지 개수)

In [31]:
result = []

for i in range(4):
    print(i + 1, '페이지 크롤링...')
    result += nike_page(i + 1)

result

1 페이지 크롤링...
2 페이지 크롤링...
3 페이지 크롤링...
4 페이지 크롤링...


[{'제품명': '나이키 에어맥스 2090',
  '가격': 169000,
  'url': 'https://www.nike.com/kr/ko_kr/t/men/fw/nike-sportswear/CT1091-101/getl12/nike-air-max-2090'},
 {'제품명': '나이키 데이브레이크',
  '가격': 119000,
  'url': 'https://www.nike.com/kr/ko_kr/t/women/fw/nike-sportswear/CK2351-007/agat92/w-nike-dbreak'},
 {'제품명': '나이키 에어맥스 97',
  '가격': 199000,
  'url': 'https://www.nike.com/kr/ko_kr/t/men/fw/nike-sportswear/921826-101/nzqc95/nike-air-max-97'},
 {'제품명': '나이키 리액트 비전',
  '가격': 149000,
  'url': 'https://www.nike.com/kr/ko_kr/t/men/fw/nike-sportswear/CD4373-103/hioo48/nike-react-vision'},
 {'제품명': '나이키 줌 플라이 3',
  '가격': 179000,
  'url': 'https://www.nike.com/kr/ko_kr/t/men/fw/running/AT8240-007/RzS0oB/zoom-fly-3'},
 {'제품명': '나이키 킬샷 OG SP',
  '가격': 109000,
  'url': 'https://www.nike.com/kr/ko_kr/t/men/fw/nike-sportswear/CU9180-100/grfg21/nike-killshot-og-sp'},
 {'제품명': '에어맥스 90 NRG',
  '가격': 169000,
  'url': 'https://www.nike.com/kr/ko_kr/t/men/fw/nike-sportswear/CK6467-001/shce69/air-max-90-nrg'},
 {'제품명': '나

In [37]:
df = pd.DataFrame(result)
df

,제품명,가격,url
0,나이키 에어맥스 2090,169000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
1,나이키 데이브레이크,119000,https://www.nike.com/kr/ko_kr/t/women/fw/nike-...
2,나이키 에어맥스 97,199000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
3,나이키 리액트 비전,149000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
4,나이키 줌 플라이 3,179000,https://www.nike.com/kr/ko_kr/t/men/fw/running...
...,...,...,...
155,나이키 에어 줌 슈퍼렙,139000,https://www.nike.com/kr/ko_kr/t/men/fw/men-tra...
156,나이키 프리 메트콘 3,139000,https://www.nike.com/kr/ko_kr/t/adult-unisex/f...
157,나이키코트 에어 줌 베이퍼 X,179000,https://www.nike.com/kr/ko_kr/t/men/fw/tennis/...
158,나이키 슈퍼렙 고,119000,https://www.nike.com/kr/ko_kr/t/men/fw/men-tra...


In [38]:
df.columns = ['상품명', '가격', '상세 페이지']
df.index = df.index + 1
df

,상품명,가격,상세 페이지
1,나이키 에어맥스 2090,169000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
2,나이키 데이브레이크,119000,https://www.nike.com/kr/ko_kr/t/women/fw/nike-...
3,나이키 에어맥스 97,199000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
4,나이키 리액트 비전,149000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
5,나이키 줌 플라이 3,179000,https://www.nike.com/kr/ko_kr/t/men/fw/running...
...,...,...,...
156,나이키 에어 줌 슈퍼렙,139000,https://www.nike.com/kr/ko_kr/t/men/fw/men-tra...
157,나이키 프리 메트콘 3,139000,https://www.nike.com/kr/ko_kr/t/adult-unisex/f...
158,나이키코트 에어 줌 베이퍼 X,179000,https://www.nike.com/kr/ko_kr/t/men/fw/tennis/...
159,나이키 슈퍼렙 고,119000,https://www.nike.com/kr/ko_kr/t/men/fw/men-tra...


In [42]:
df.sort_values(by=['가격'], axis=0, ascending=False) #ascending=False 내림차순

,상품명,가격,상세 페이지
130,나이키 어댑트 오토 맥스,439000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
11,나이키 줌X 베이퍼플라이 넥스트%,299000,https://www.nike.com/kr/ko_kr/t/adult-unisex/f...
32,르브론 17 FP,269000,https://www.nike.com/kr/ko_kr/t/men/fw/basketb...
86,나이키 에어 베이퍼맥스 360,259000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
13,나이키 에어 베이퍼맥스 2020 FK,259000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
...,...,...,...
139,나이키 프리 런 5.0 2020,76300,https://www.nike.com/kr/ko_kr/t/men/fw/running...
37,나이키 줌 2K,69300,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
151,조던 하이드로 8,69000,https://www.nike.com/kr/ko_kr/t/men/fw/basketb...
155,나이키 베나시 JDI 슬라이드,39000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...


In [47]:
df.sort_values(by=['가격'], axis=0, ascending=False, inplace=True) #ascending=False 내림차순 inplace=True 원본 변경
df

,상품명,가격,상세 페이지
130,나이키 어댑트 오토 맥스,439000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
11,나이키 줌X 베이퍼플라이 넥스트%,299000,https://www.nike.com/kr/ko_kr/t/adult-unisex/f...
32,르브론 17 FP,269000,https://www.nike.com/kr/ko_kr/t/men/fw/basketb...
86,나이키 에어 베이퍼맥스 360,259000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
13,나이키 에어 베이퍼맥스 2020 FK,259000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
...,...,...,...
139,나이키 프리 런 5.0 2020,76300,https://www.nike.com/kr/ko_kr/t/men/fw/running...
37,나이키 줌 2K,69300,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
151,조던 하이드로 8,69000,https://www.nike.com/kr/ko_kr/t/men/fw/basketb...
155,나이키 베나시 JDI 슬라이드,39000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...


In [48]:
df = df.reset_index(drop = True)
df

,상품명,가격,상세 페이지
0,나이키 어댑트 오토 맥스,439000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
1,나이키 줌X 베이퍼플라이 넥스트%,299000,https://www.nike.com/kr/ko_kr/t/adult-unisex/f...
2,르브론 17 FP,269000,https://www.nike.com/kr/ko_kr/t/men/fw/basketb...
3,나이키 에어 베이퍼맥스 360,259000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
4,나이키 에어 베이퍼맥스 2020 FK,259000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
...,...,...,...
155,나이키 프리 런 5.0 2020,76300,https://www.nike.com/kr/ko_kr/t/men/fw/running...
156,나이키 줌 2K,69300,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
157,조던 하이드로 8,69000,https://www.nike.com/kr/ko_kr/t/men/fw/basketb...
158,나이키 베나시 JDI 슬라이드,39000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...


In [49]:
df.index = df.index + 1

In [50]:
df

,상품명,가격,상세 페이지
1,나이키 어댑트 오토 맥스,439000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
2,나이키 줌X 베이퍼플라이 넥스트%,299000,https://www.nike.com/kr/ko_kr/t/adult-unisex/f...
3,르브론 17 FP,269000,https://www.nike.com/kr/ko_kr/t/men/fw/basketb...
4,나이키 에어 베이퍼맥스 360,259000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
5,나이키 에어 베이퍼맥스 2020 FK,259000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
...,...,...,...
156,나이키 프리 런 5.0 2020,76300,https://www.nike.com/kr/ko_kr/t/men/fw/running...
157,나이키 줌 2K,69300,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...
158,조던 하이드로 8,69000,https://www.nike.com/kr/ko_kr/t/men/fw/basketb...
159,나이키 베나시 JDI 슬라이드,39000,https://www.nike.com/kr/ko_kr/t/men/fw/nike-sp...


In [51]:
df.to_excel("./data/nike.xlsx")

In [52]:
import os
filepath = os.path.join(os.getcwd(), 'data', '나이키신상.xlsx')
filepath

'D:\\ds2007\\data\\나이키신상.xlsx'

In [54]:
df.to_excel(filepath, encoding='euc-kr')